In [ ]:
import pandas as pd
import jieba
from sklearn.metrics import jaccard_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn.functional as F
import os 

In [ ]:
# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

In [ ]:
#读取文件
data_original = pd.read_excel('../input folder/公牛原帖_作者id合并后.xlsx')

## step 1 处理源数据

#### tips： 第一轮循环后，需要将data_original换成remaining_data

In [ ]:
# 第一轮循环后，需要将data_original换成remaining_data
split_data = data_original

### 1.1 随机抽样300条数据，交个GPT

#### tips: 每一轮的random_state应该变化，并且记录下seed的数字，方便复现

In [ ]:
#（待选
df_sampled = split_data.sample(n=300,random_state = 42)
# 查看特定列
df_content = df_sampled['作品正文内容']
df_content.to_csv('ym-loop-0-RANDOM-seed42.txt', index=False, header=None)

### 1.2 检查数据

In [ ]:
column_headers = split_data.columns.tolist()
column_headers

### 1.3 处理对应列的文本

In [ ]:
#清洗数据，删除重复值和空值
clean_data = split_data.dropna(subset=['作品标题', '作品正文内容'])

In [ ]:
# 对评论进行分词处理
## 加载停用词文件
def load_stopwords(paths):
    stopwords = set()
    for path in paths:
        with open(path, 'r', encoding='utf-8') as file:
            stopwords.update(line.strip() for line in file)
    return stopwords
stopwords_paths = ["../中文停用词/cn_stopwords.txt",
                       "../中文停用词/hit_stopwords.txt",
                       "../中文停用词/scu_stopwords.txt",
                       "../中文停用词/baidu_stopwords.txt"]
stopwords = load_stopwords(stopwords_paths)

In [ ]:
# 定义分词和过滤函数
def tokenize_and_filter(text, stopwords):
    words = jieba.cut(text)
    return [word for word in words if word not in stopwords and word.strip()]


data_fenci = clean_data['作品正文内容'].apply(lambda x: tokenize_and_filter(x, stopwords))

In [ ]:
comments = data_fenci
comments

## step 2 ：引入GPT生成的人群和特征词

In [ ]:
# 引用参考人群聚类文档
excel_path = "../1029ym论文人群关键词.xlsx"
def read_dimensions_from_excel(excel_path):
    df = pd.read_excel(excel_path)
    dimensions = {}
    for column in df.columns:
        dimensions[column] = df[column].dropna().tolist()
    return dimensions
# cluster_reference = pd.read_excel('../人群分词.xlsx')
dimensions = read_dimensions_from_excel(excel_path)
dimensions

### 2.1 引入腾讯近义词向量库

In [ ]:
# 加载腾讯AI实验室的词向量模型

word_vectors = KeyedVectors.load_word2vec_format('../Tencent Embedding/tencent-ailab-embedding-zh-d200-v0.2.0.txt', binary=False)

## step 3: 处理词向量以及相似度计算

### 3.1 GPT中的特征词转换成向量

In [ ]:
# 创建一个新的字典，用于存储每个维度下特征词的向量张量
dimensions_vectors = {}

for dimension, keywords in dimensions.items():
    # 存储当前维度下的所有特征词向量张量
    vectors = []
    for keyword in keywords:
        # 检查词向量模型中是否有该词
        if keyword in word_vectors.key_to_index:
            # 将词向量转换为PyTorch张量，并加载到GPU
            vector_tensor = torch.tensor(word_vectors[keyword], dtype=torch.float32, device=device)
            vectors.append(vector_tensor)
    # 将转换后的向量张量列表存储在新字典中
    dimensions_vectors[dimension] = vectors

In [ ]:
# 设计聚类的名称
def read_cluster_names(excel_path):
    df = pd.read_excel(excel_path)
    return df.columns.tolist()  # 假设聚类名在第一行
cluster_names = read_cluster_names(excel_path)

### 3.2 对每条数据进行打分

#### 3.2.1 对每条数据进行维度归纳

In [ ]:
def calculate_weighted_similarity_with_threshold_and_words(comment_words, dimensions_vectors, word_vectors, threshold=0.55, device='cuda'):
    """
    计算评论中每个词的相似度，使用阈值过滤不相关词，并进行加权计算，
    返回每个评论的相似度分数（基于维度特征词的加权平均）和最相关的特征词。
    """
    comment_vectors = [get_word_tensor(word, word_vectors, device) for word in comment_words if word in word_vectors]
    if not comment_vectors:
        return {}, []  # 返回空的相似度和空的词列表

    comment_matrix = torch.stack(comment_vectors)
    scores = {}
    related_words = []

    for dimension, feature_vectors in dimensions_vectors.items():
        total_weight = 0.0
        weighted_similarity_sum = 0.0
        word_similarities = []

        for fv in feature_vectors:
            sims = torch.stack([F.cosine_similarity(comment_matrix, fv.unsqueeze(0).expand_as(comment_matrix), dim=1)])
            similarity = sims.max().item() if sims.numel() > 0 else 0

            # 如果相似度大于阈值，进行加权
            if similarity > threshold:
                weight = similarity
                weighted_similarity_sum += similarity * weight
                total_weight += weight
                word_similarities.append((fv, similarity))  # 存储词向量和相似度

        if total_weight > 0:
            average_similarity = weighted_similarity_sum / total_weight
            scores[dimension] = average_similarity

            # 按照相似度降序排序，并选择前三个词
            word_similarities.sort(key=lambda x: x[1], reverse=True)
            top_words = [word_vectors.index_to_key[word_similarities[i][0].tolist()] for i in range(min(3, len(word_similarities)))]
            related_words.append(top_words)
        else:
            scores[dimension] = 0
            related_words.append([])

    return scores, related_words



In [ ]:
# 计算每条评论的相似度得分和最相关的词
similarity_scores_subset = comments.apply(lambda x: calculate_weighted_similarity_with_threshold_and_words(x, dimensions_vectors, word_vectors, device='cuda', threshold=0.5))

# 转化成DataFrame，存储相似度得分
similarity_scores_df = pd.DataFrame([score[0] for score in similarity_scores_subset.tolist()])

# 提取每条评论最相关的词
related_words_subset = [score[1] for score in similarity_scores_subset.tolist()]
related_words_df = pd.DataFrame(related_words_subset, columns=["Top_1", "Top_2", "Top_3"])

# 计算每条评论的最大相似度得分
max_similarity_scores = similarity_scores_df.max(axis=1)

# 设置阈值
threshold = 0.55

# 基于阈值筛选评论，得到符合条件的评论索引
filtered_indexes = max_similarity_scores[max_similarity_scores > threshold].index
filtered_similarity_scores_df = similarity_scores_df.loc[filtered_indexes]
filtered_related_words_df = related_words_df.loc[filtered_indexes]

# 选择每个评论最高分的维度（每条评论与哪个主题最相关）
scores_max = filtered_similarity_scores_df.max(axis=1)
max_scores = filtered_similarity_scores_df.idxmax(axis=1)

# 输出原始功能的结果
print("\nFiltered Similarity Scores:")
print(filtered_similarity_scores_df)

print("\nFiltered Related Words (Top 3):")
print(filtered_related_words_df)

print("\nMax Scores per Comment:")
print(scores_max)

print("\nMax Scores (Topic) per Comment:")
print(max_scores)


In [ ]:
# 计算每条评论的相似度得分
similarity_scores_subset = comments.apply(lambda x: calculate_weighted_similarity_with_threshold_and_words(x, dimensions_vectors, word_vectors, device='cuda', threshold=0.5))

# 转化成DataFrame
similarity_scores_df = pd.DataFrame(similarity_scores_subset.tolist())

# 计算每条评论的最大相似度得分
max_similarity_scores = similarity_scores_df.max(axis=1)

# 设置阈值
threshold = 0.55

# 基于阈值筛选评论
filtered_indexes = max_similarity_scores[max_similarity_scores > threshold].index
filtered_similarity_scores_df = similarity_scores_df.loc[filtered_indexes]

# 选择每个评论最高分的维度
scores_max = filtered_similarity_scores_df.max(axis=1)
max_scores = filtered_similarity_scores_df.idxmax(axis=1)

# 输出原始功能的结果
print("\nFiltered Similarity Scores:")
print(filtered_similarity_scores_df)
print("\nMax Scores per Comment:")
print(scores_max)
print("\nMax Scores (Topic) per Comment:")
print(max_scores)

### 3.2.2 针对每组数据计算，并制作citespace里要用到的图

In [ ]:
# 获取每条评论的相似度得分和与主题相关的前三个词
def get_top_words_for_comments(comments, dimensions_vectors, word_vectors, device='cuda', threshold=0.55):
    top_words_per_comment = []
    similarity_scores_per_comment = []
    
    for comment in comments:
        scores, top_words = calculate_weighted_similarity_with_threshold_and_words(comment, dimensions_vectors, word_vectors, threshold=threshold, device=device)
        top_words_per_comment.append(top_words)
        similarity_scores_per_comment.append(scores)
    
    return top_words_per_comment, similarity_scores_per_comment

top_words_per_comment, similarity_scores_per_comment = get_top_words_for_comments(comments, dimensions_vectors, word_vectors, device='cuda', threshold=0.55)

# 输出与每个评论相关的前三个词
for idx, top_words in enumerate(top_words_per_comment):
    print(f"Comment {idx + 1}:")
    print("Top related words:", top_words)
    print()

In [ ]:
# 调用计算和逆向映射函数
similarity_scores_subset, top_related_words = extract_top_related_words_for_comments(
    comments, dimensions_vectors, word_vectors, device='cuda', threshold=0.55, top_n=3
)

# 展平 top_related_words 中的词列表
top_related_words_flat = [word for sublist in top_related_words for word in sublist]

# 调用逆向映射函数，获取最终的相关词
final_related_words = reverse_mapping_to_original_words(top_related_words_flat, comments[0], word_vectors, threshold=0.55)

print(f"Final Related Words: {final_related_words}")


In [ ]:
print(f"Top Related Words (Before Reverse Mapping): {top_related_words}")
print(f"Comments: {comments}")

In [ ]:
# 整合数据，生成CiteSpace所需的格式
citespace_data = []
for index, row in top_related_words_df.iterrows():
    comment_similarity_scores = similarity_scores_df.iloc[index].to_dict()
    for word in row:
        word_info = {
            "Keyword": word, 
            "Frequency": row.value_counts().get(word, 0),  # 计算频率
            "Related Dimension": comment_similarity_scores,  # 维度的相关性
        }
        citespace_data.append(word_info)

# 转换为DataFrame
citespace_df = pd.DataFrame(citespace_data)

# 保存为csv格式，供CiteSpace使用
citespace_df.to_csv('citespace_input.csv', index=False)

# step 4：导出相应的数据

## 4.1 已经筛选出来的数据，以及还有多少数据未被筛选出来

In [ ]:
# 获取不符合条件的数据
remaining_indexes = max_similarity_scores[max_similarity_scores <= threshold].index
remaining_similarity_scores_df = similarity_scores_df.loc[remaining_indexes]

# 获取筛选出的数据和未筛选的数据
filtered_data = comments.loc[filtered_indexes]
remaining_data = comments.loc[remaining_indexes]

# 打印未筛选数据的数量
print(f"未筛选数据的数量：{len(remaining_data)}")

## 4.2 将已经筛选出来的数据保存到datafarame中

In [ ]:
# 第一轮循环
# 保存已筛选的数据到DataFrame
filtered_data_df = pd.DataFrame({
    'Comment': filtered_data,
    'Max Similarity Score': max_similarity_scores.loc[filtered_indexes],  # 最大相似度得分
    'Matching Dimension': filtered_similarity_scores_df.idxmax(axis=1)  # 匹配的主题维度
})

In [ ]:
# 第一轮循环
# 导出筛选后的数据到Excel
filtered_data_df.to_excel('../output_folder/筛选后的数据.xlsx', index=False)

In [ ]:
from openpyxl import load_workbook
# 后续循环
# 1. 构造新的 DataFrame
new_filtered_data_df = pd.DataFrame({
    'Comment': filtered_data,
    'Max Similarity Score': max_similarity_scores.loc[filtered_indexes],  # 最大相似度得分
    'Matching Dimension': filtered_similarity_scores_df.idxmax(axis=1)
})

# 2. 读取现有的 Excel 文件
excel_path = '../output_folder/筛选后的数据.xlsx'

try:
    # 尝试读取现有的文件
    book = load_workbook(excel_path)
except FileNotFoundError:
    # 如果文件不存在，创建一个新的文件
    book = None

# 3. 使用 ExcelWriter 添加到 Sheet2
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    if book:
        # 如果文件已经存在，加载现有的工作簿
        writer.book = book
        
    # 将新数据添加到 Sheet2 中（第几轮循环就写Sheet X 号）
    new_filtered_data_df.to_excel(writer, index=False, sheet_name='Sheet2')

    # 打印输出确认
    print(f"新数据已成功添加到 {excel_path} 的 Sheet2 中")



### 4.2.1 导出本次循环中对应人群的帖子

In [ ]:
# 确定保存文件的基础路径
base_path = '../output_folder/已经筛选出的人群的帖子'

# 检查并创建基础路径
if not os.path.exists(base_path):
    os.makedirs(base_path)

# 按 'Matching Dimension' 进行分组
for dimension, group_df in filtered_data_df.groupby('Matching Dimension'):
    # 构建文件名
    filename = f"主题_{dimension}.xlsx"
    file_path = os.path.join(base_path, filename)
    
    # 保存到Excel文件
    group_df.to_excel(file_path, index=False)
    
    print(f"Saved {dimension} posts to {file_path}")

### 4.2.2 保存以及导出citespace所需要的网络图数据

In [ ]:
# 第一轮循环
excel_path = '../output_folder/citespace_network_data.xlsx'

with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    # 保存节点数据到 'Nodes' 工作表
    nodes_df.to_excel(writer, index=False, sheet_name='Nodes')
    
    # 保存边数据到 'Edges' 工作表
    edges_df.to_excel(writer, index=False, sheet_name='Edges')
    
    print(f"节点和边数据已保存到 {excel_path}")

In [ ]:
# 后续循环
# Excel 文件路径
excel_path = '../output_folder/citespace_network_data.xlsx'

# 读取现有的工作簿
try:
    # 尝试加载现有文件
    book = load_workbook(excel_path)
except FileNotFoundError:
    # 如果文件不存在，创建一个新的文件
    book = None

# 使用 ExcelWriter 将数据追加到现有文件
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    if book:
        # 如果文件已存在，加载工作簿并将其赋给 writer
        writer.book = book

    # 将新数据追加到 'Nodes' 工作表（如果已存在，则会在现有数据后追加）
    nodes_df.to_excel(writer, index=False, sheet_name='Nodes', header=not book)

    # 将新数据追加到 'Edges' 工作表（如果已存在，则会在现有数据后追加）
    edges_df.to_excel(writer, index=False, sheet_name='Edges', header=not book)

    print(f"新数据已追加到 {excel_path}")